Tp 4: : Spark MLlib


In [2]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=ab7c4a3e913095dc39d7831c2c2e7ae9180bebef085aa0013ddbf6c4fd02d505
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("tp4").getOrCreate()

DataFrame = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load("/content/*.csv")

2/ Afficher le schéma du Dataframe obtenu

In [5]:
DataFrame.printSchema()
#DataFrame.collect()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



3/Remplir les valeurs manquantes (NaN) avec la valeur 0.

4/ Ajouter une nouvelle colonne nommée "day_of_week". La valeur de la colonne est le jour de la semaine correspondant à la date de chaque ligne dans la colonne "InvoiceDate"

In [6]:
from pyspark.sql.functions import date_format, col
PreparingData = DataFrame.na.fill(0).withColumn("day_of_week", date_format(col("InvoiceDate"), "EEEE"))

5/ Diviser les données en un ensemble d'apprentissage et un ensemble de test. Effectuer
la division en se basant sur l'attribut InvoiceDate : l'ensemble d'apprentissage contient
les achats effectués avant 2010-12-13 et l'ensemble de test contient les achats effectués
durant ou après 2010-12-13.

In [7]:
apprentData = PreparingData.where("InvoiceDate < '2010-12-13'")
testData =  PreparingData.where("InvoiceDate >= '2010-12-13'")

print(apprentData.count())
print(testData.count())

26732
18676


6/Créer un StringIndexer qui permet de transformer les jours de semaine présents dans
la colonne day_of_week en valeurs numériques correspondantes.

In [8]:
from pyspark.ml.feature import StringIndexer
numeroter = StringIndexer().setInputCol("day_of_week").setOutputCol("day_of_week_num")

7/En utilisant le StringIndexer, Spark peut par exemple représenter samedi par 6 et
lundi par 1. Cependant, avec ce schéma de numérotation, nous indiquons
implicitement que samedi est supérieur à lundi (par des valeurs numériques pures).
Ceci est évidemment incorrect. Comment résoudre ce problème ?

In [9]:
from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder().setInputCol("day_of_week_num").setOutputCol("day_of_week_encoded")

8/Créer un VectorAssembler contenant trois attributs : UnitPrice, Quantity, et
day_of_week_encoded.

In [10]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler().setInputCols(["UnitPrice", "Quantity", "day_of_week_encoded"]).setOutputCol("features")

9/Créer un pipeline configuré avec les résultats des étapes 6, 7 et 8

In [11]:
from pyspark.ml import Pipeline
transformationPipeline = Pipeline()\
.setStages([numeroter, encoder, vectorAssembler])

10)	Notre StringIndexer doit savoir combien de valeurs uniques il y a à indexer, comment résoudre ce problème ?

In [12]:
AdaptedPipeline = transformationPipeline.fit(apprentData)

11)	Transformer les données de l’ensemble d’apprentissage en se basant sur les étapes (stages) du pipeline.

In [13]:
transforApprent = AdaptedPipeline.transform(apprentData)

12)	Créer une instance de KMeans, on supposse que le nombre de clusters est 20.

In [14]:
from pyspark.ml.clustering import KMeans
kmeans = KMeans().setK(20)

13)	Lancer l’apprentissage de KMeans en se basant sur l’ensemble obtenu dans l’étape 11.

In [15]:
kmeansModel = kmeans.fit(transforApprent)

14)	Effectuer des prédictions sur l’ensemble de test.

In [17]:
transTest = AdaptedPipeline.transform(testData)
predictions = kmeansModel.transform(transTest)

15)	Calculer le coefficient de silhouette.

In [18]:
from pyspark.ml.evaluation import ClusteringEvaluator
evaluation = ClusteringEvaluator()
silhouette = evaluation.evaluate(predictions)